In [1]:
!pip install neat-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [17]:
import pandas as pd
import neat
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data1 = pd.read_csv('/Users/macbook/Desktop/ВУЗ/Практикум по программированию/lab_2-5/lab_2/dataset_1.csv')
data2 = pd.read_csv('/Users/macbook/Desktop/ВУЗ/Практикум по программированию/lab_2-5/lab_2/dataset_12.csv')

def encode_categorical(data):
    categorical_cols = data.select_dtypes(include=['object']).columns  
    data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)  
    return data_encoded

data1_encoded = encode_categorical(data1)
data2_encoded = encode_categorical(data2)

X1 = data1_encoded.iloc[:, :-1].values 
y1 = data1_encoded.iloc[:, -1].values 

X2 = data2_encoded.iloc[:, :-1].values
y2 = data2_encoded.iloc[:, -1].values

def prepare_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

X1_train, X1_test, y1_train, y1_test, scaler1 = prepare_data(X1, y1)
X2_train, X2_test, y2_train, y2_test, scaler2 = prepare_data(X2, y2)

def eval_genomes_hyperneat(genomes, config, X_train, y_train):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        correct = 0
        for xi, target in zip(X_train, y_train):
            output = net.activate(xi)
            if np.argmax(output) == target:
                correct += 1
        genome.fitness = correct / len(y_train)

def run_hyperneat(eval_fn, config_path, X_train, y_train, X_test, y_test):
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )

    pop = neat.Population(config)
    pop.add_reporter(neat.StdOutReporter(True))
    winner = pop.run(lambda g, c: eval_fn(g, c, X_train, y_train), 50)

    net = neat.nn.FeedForwardNetwork.create(winner, config)
    correct = 0
    for xi, yi in zip(X_test, y_test):
        output = net.activate(xi)
        if np.argmax(output) == yi:
            correct += 1
    print("\nTest Accuracy:", correct / len(y_test))

print("=== Маленький датасет ===")
run_hyperneat(eval_genomes_hyperneat, 'Config-logreg-hyperneat.cfg', X1_train, y1_train, X1_test, y1_test)

print("\n=== Большой датасет ===")
run_hyperneat(eval_genomes_hyperneat, 'Config-tree-hyperneat.cfg', X2_train, y2_train, X2_test, y2_test)


=== Маленький датасет ===

 ****** Running generation 0 ****** 

Population's average fitness: 0.80000 stdev: 0.00000
Best fitness: 0.80000 - size: (1, 19) - species 1 - id 1
Average adjusted fitness: -0.000
Mean genetic distance 2.119, standard deviation 0.764
Population of 50 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    41      0.8   -0.000     0
     2    0     9       --       --     0
Total extinctions: 0
Generation time: 0.012 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.80000 stdev: 0.00000
Best fitness: 0.80000 - size: (1, 19) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 2.532, standard deviation 0.994
Population of 50 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    40      0.8    0.000     1
     2    1     5      0.8   -0.000     0
     3    0     5       --       --     0